In [ ]:
-- Configuration Variables
DECLARE @BackupPath NVARCHAR(256) = N'C:\SQLBackups\RomanEmpire\'
DECLARE @DatabaseName NVARCHAR(50) = N'RomanEmpire'
DECLARE @RetentionDays INT = 30

-- Create Backup Directory If Not Exists
EXEC master.dbo.xp_create_subdir @BackupPath
GO
-- 1. Full Database Backup
CREATE OR ALTER PROCEDURE dbo.usp_FullBackup
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @BackupFile NVARCHAR(256)
    DECLARE @BackupName NVARCHAR(100)
    
    SET @BackupFile = @BackupPath + @DatabaseName + '_FULL_' + 
        CONVERT(NVARCHAR(8), GETDATE(), 112) + '_' + 
        REPLACE(CONVERT(NVARCHAR(8), GETDATE(), 108), ':', '') + '.bak'
    
    SET @BackupName = @DatabaseName + ' Full Backup - ' + 
        CONVERT(NVARCHAR(20), GETDATE(), 120)
    
    BACKUP DATABASE @DatabaseName 
    TO DISK = @BackupFile
    WITH COMPRESSION, 
         CHECKSUM,
         INIT,
         NAME = @BackupName,
         STATS = 10
    
    -- Verify Backup
    RESTORE VERIFYONLY FROM DISK = @BackupFile
END
GO

-- 2. Differential Backup
CREATE OR ALTER PROCEDURE dbo.usp_DifferentialBackup
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @BackupFile NVARCHAR(256)
    DECLARE @BackupName NVARCHAR(100)
    
    SET @BackupFile = @BackupPath + @DatabaseName + '_DIFF_' + 
        CONVERT(NVARCHAR(8), GETDATE(), 112) + '_' + 
        REPLACE(CONVERT(NVARCHAR(8), GETDATE(), 108), ':', '') + '.bak'
    
    SET @BackupName = @DatabaseName + ' Differential Backup - ' + 
        CONVERT(NVARCHAR(20), GETDATE(), 120)
    
    BACKUP DATABASE @DatabaseName 
    TO DISK = @BackupFile
    WITH DIFFERENTIAL,
         COMPRESSION,
         CHECKSUM,
         INIT,
         NAME = @BackupName,
         STATS = 10
    
    -- Verify Backup
    RESTORE VERIFYONLY FROM DISK = @BackupFile
END
GO

-- 3. Transaction Log Backup
CREATE OR ALTER PROCEDURE dbo.usp_LogBackup
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @BackupFile NVARCHAR(256)
    DECLARE @BackupName NVARCHAR(100)
    
    SET @BackupFile = @BackupPath + @DatabaseName + '_LOG_' + 
        CONVERT(NVARCHAR(8), GETDATE(), 112) + '_' + 
        REPLACE(CONVERT(NVARCHAR(8), GETDATE(), 108), ':', '') + '.trn'
    
    SET @BackupName = @DatabaseName + ' Log Backup - ' + 
        CONVERT(NVARCHAR(20), GETDATE(), 120)
    
    BACKUP LOG @DatabaseName 
    TO DISK = @BackupFile
    WITH COMPRESSION,
         CHECKSUM,
         INIT,
         NAME = @BackupName,
         STATS = 10
    
    -- Verify Backup
    RESTORE VERIFYONLY FROM DISK = @BackupFile
END
GO

-- 4. Cleanup Old Backups
CREATE OR ALTER PROCEDURE dbo.usp_CleanupOldBackups
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Create temporary table to store files to delete
    CREATE TABLE #FilesToDelete (
        FileName NVARCHAR(256)
    )
    
    -- Get list of old backup files
    INSERT INTO #FilesToDelete
    EXEC xp_cmdshell 'dir /B /A-D C:\SQLBackups\RomanEmpire\*.bak'
    
    -- Delete files older than retention period
    DECLARE @cmd NVARCHAR(512)
    DECLARE @file NVARCHAR(256)
    
    DECLARE file_cursor CURSOR FOR 
    SELECT FileName 
    FROM #FilesToDelete 
    WHERE FileName IS NOT NULL
        AND CAST(SUBSTRING(FileName, CHARINDEX('_', FileName) + 1, 8) AS DATE) 
            < DATEADD(DAY, -@RetentionDays, GETDATE())
    
    OPEN file_cursor
    FETCH NEXT FROM file_cursor INTO @file
    
    WHILE @@FETCH_STATUS = 0
    BEGIN
        SET @cmd = 'DEL "' + @BackupPath + @file + '"'
        EXEC xp_cmdshell @cmd
        
        FETCH NEXT FROM file_cursor INTO @file
    END
    
    CLOSE file_cursor
    DEALLOCATE file_cursor
    
    DROP TABLE #FilesToDelete
END
GO

-- 5. Point-in-Time Recovery
CREATE OR ALTER PROCEDURE dbo.usp_PointInTimeRecovery
    @RestorePoint DATETIME,
    @NewDatabaseName NVARCHAR(50) = NULL
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @LastFullBackup NVARCHAR(256)
    DECLARE @LastDiffBackup NVARCHAR(256)
    DECLARE @TargetDatabase NVARCHAR(50)
    
    SET @TargetDatabase = ISNULL(@NewDatabaseName, @DatabaseName)
    
    -- Find last full backup before restore point
    SELECT TOP 1 @LastFullBackup = BackupFile
    FROM (
        SELECT 
            physical_device_name AS BackupFile,
            backup_start_date,
            backup_type = CASE type 
                WHEN 'D' THEN 'Full'
                WHEN 'I' THEN 'Differential'
                WHEN 'L' THEN 'Log'
            END
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m 
            ON b.media_set_id = m.media_set_id
        WHERE database_name = @DatabaseName
            AND type = 'D'
            AND backup_start_date <= @RestorePoint
    ) AS Backups
    ORDER BY backup_start_date DESC
    
    -- Restore Full Backup
    RESTORE DATABASE @TargetDatabase
    FROM DISK = @LastFullBackup
    WITH NORECOVERY,
         REPLACE,
         STATS = 10
    
    -- Find and restore differential backup if exists
    SELECT TOP 1 @LastDiffBackup = BackupFile
    FROM (
        SELECT 
            physical_device_name AS BackupFile,
            backup_start_date
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m 
            ON b.media_set_id = m.media_set_id
        WHERE database_name = @DatabaseName
            AND type = 'I'
            AND backup_start_date <= @RestorePoint
            AND backup_start_date > (
                SELECT MAX(backup_start_date)
                FROM msdb.dbo.backupset
                WHERE database_name = @DatabaseName
                    AND type = 'D'
                    AND backup_start_date <= @RestorePoint
            )
    ) AS DiffBackups
    ORDER BY backup_start_date DESC
    
    IF @LastDiffBackup IS NOT NULL
    BEGIN
        RESTORE DATABASE @TargetDatabase
        FROM DISK = @LastDiffBackup
        WITH NORECOVERY,
             STATS = 10
    END
    
    -- Restore Transaction Logs
    DECLARE @LogBackup NVARCHAR(256)
    DECLARE log_cursor CURSOR FOR
        SELECT physical_device_name
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m 
            ON b.media_set_id = m.media_set_id
        WHERE database_name = @DatabaseName
            AND type = 'L'
            AND backup_start_date > COALESCE(
                (SELECT MAX(backup_start_date)
                 FROM msdb.dbo.backupset
                 WHERE database_name = @DatabaseName
                    AND type = 'I'
                    AND backup_start_date <= @RestorePoint),
                (SELECT MAX(backup_start_date)
                 FROM msdb.dbo.backupset
                 WHERE database_name = @DatabaseName
                    AND type = 'D'
                    AND backup_start_date <= @RestorePoint)
            )
            AND backup_start_date <= @RestorePoint
        ORDER BY backup_start_date
    
    OPEN log_cursor
    FETCH NEXT FROM log_cursor INTO @LogBackup
    
    WHILE @@FETCH_STATUS = 0
    BEGIN
        RESTORE LOG @TargetDatabase
        FROM DISK = @LogBackup
        WITH NORECOVERY,
             STATS = 10
        
        FETCH NEXT FROM log_cursor INTO @LogBackup
    END
    
    CLOSE log_cursor
    DEALLOCATE log_cursor
    
    -- Recover database
    RESTORE DATABASE @TargetDatabase WITH RECOVERY
END
GO

-- 6. Create Backup Jobs
-- Note: These would typically be created as SQL Server Agent Jobs
-- Example schedule:
-- Full backup: Weekly (Sunday at 12:00 AM)
-- Differential backup: Daily (at 12:00 AM except Sunday)
-- Transaction log backup: Every 15 minutes
-- Cleanup: Weekly (Sunday at 2:00 AM)

USE msdb
GO

-- Weekly Full Backup Job
EXEC dbo.sp_add_job
    @job_name = N'RomanEmpire - Weekly Full Backup',
    @enabled = 1,
    @description = N'Weekly full backup of RomanEmpire database'

EXEC sp_add_jobstep
    @job_name = N'RomanEmpire - Weekly Full Backup',
    @step_name = N'Execute Full Backup',
    @subsystem = N'TSQL',
    @command = N'EXEC dbo.usp_FullBackup'

EXEC dbo.sp_add_schedule
    @schedule_name = N'Weekly Sunday Midnight',
    @freq_type = 8, -- Weekly
    @freq_interval = 1, -- Sunday
    @freq_recurrence_factor = 1,
    @active_start_time = 000000

EXEC sp_attach_schedule
    @job_name = N'RomanEmpire - Weekly Full Backup',
    @schedule_name = N'Weekly Sunday Midnight'

-- Daily Differential Backup Job (similar structure)
-- Transaction Log Backup Job (similar structure)
-- Weekly Cleanup Job (similar structure)

In [ ]:
-- Create database roles representing Roman hierarchy
USE RomanEmpire;
GO

-- 1. Create roles
CREATE ROLE [Emperor]; -- Highest authority
CREATE ROLE [Consul]; -- Senior administrator
CREATE ROLE [Praetor]; -- Legal authority
CREATE ROLE [Censor]; -- Financial oversight
CREATE ROLE [Tribune]; -- Provincial representative
CREATE ROLE [Quaestor]; -- Treasury official
CREATE ROLE [Aedile]; -- Public works manager
CREATE ROLE [Scribe]; -- Read-only access

-- 2. Role Hierarchy and Permissions

-- Emperor (complete control)
ALTER AUTHORIZATION ON DATABASE::RomanEmpire TO [Emperor];
GRANT CONTROL ON DATABASE::RomanEmpire TO [Emperor];

-- Consul (administrative rights)
GRANT ALTER ANY USER TO [Consul];
GRANT ALTER ANY ROLE TO [Consul];
GRANT CREATE PROCEDURE TO [Consul];
GRANT CREATE TABLE TO [Consul];
GRANT CREATE VIEW TO [Consul];

-- Praetor (legal administration)
GRANT EXECUTE ON SCHEMA::[Legal] TO [Praetor];
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::[Legal] TO [Praetor];
GRANT SELECT ON SCHEMA::[Financial] TO [Praetor];

-- Censor (financial oversight)
GRANT SELECT, UPDATE ON SCHEMA::[Financial] TO [Censor];
GRANT EXECUTE ON OBJECT::Financial.CalculateProvincialTax TO [Censor];
GRANT EXECUTE ON OBJECT::Financial.AuditTaxCollection TO [Censor];

-- Tribune (provincial management)
GRANT SELECT, INSERT, UPDATE ON SCHEMA::[Provincial] TO [Tribune];
GRANT EXECUTE ON OBJECT::Provincial.ManagePopulation TO [Tribune];
GRANT EXECUTE ON OBJECT::Provincial.ReportTaxes TO [Tribune];

-- Quaestor (treasury operations)
GRANT SELECT, INSERT ON SCHEMA::[Financial] TO [Quaestor];
GRANT EXECUTE ON OBJECT::Financial.RecordTaxCollection TO [Quaestor];
GRANT EXECUTE ON OBJECT::Financial.GenerateFinancialReport TO [Quaestor];

-- Aedile (public works)
GRANT SELECT, INSERT, UPDATE ON SCHEMA::[PublicWorks] TO [Aedile];
GRANT EXECUTE ON OBJECT::PublicWorks.ManageConstruction TO [Aedile];
GRANT EXECUTE ON OBJECT::PublicWorks.AllocateResources TO [Aedile];

-- Scribe (read-only access)
GRANT SELECT ON DATABASE::RomanEmpire TO [Scribe];
DENY INSERT, UPDATE, DELETE ON DATABASE::RomanEmpire TO [Scribe];
Go
-- 3. Create schemas for organization
CREATE SCHEMA Legal;
GO
CREATE SCHEMA Financial;
Go
CREATE SCHEMA Provincial;
GO
CREATE SCHEMA PublicWorks;
GO

-- 4. Example stored procedures with role-based security

-- Financial procedures
CREATE OR ALTER PROCEDURE Financial.RecordTaxCollection
    @ProvinceID INT,
    @Amount DECIMAL(18,2)
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Security check
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.database_principals dp
        JOIN sys.database_role_members rm ON dp.principal_id = rm.member_principal_id
        WHERE dp.name = SYSTEM_USER 
        AND EXISTS (
            SELECT 1 
            FROM sys.database_principals rp
            WHERE rp.principal_id = rm.role_principal_id 
            AND rp.name IN ('Emperor', 'Consul', 'Quaestor')
        )
    )
    BEGIN
        RAISERROR('Access denied. Requires Emperor, Consul, or Quaestor role.', 16, 1);
        RETURN;
    END
    
    -- Record tax collection
    INSERT INTO Financial.TaxRecords (ProvinceID, Amount, CollectionDate, RecordedBy)
    VALUES (@ProvinceID, @Amount, GETDATE(), SYSTEM_USER);
END;
GO

-- Provincial management
CREATE OR ALTER PROCEDURE Provincial.ManagePopulation
    @ProvinceID INT,
    @Population INT,
    @CensusDate DATE
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Security check
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.database_principals dp
        JOIN sys.database_role_members rm ON dp.principal_id = rm.member_principal_id
        WHERE dp.name = SYSTEM_USER 
        AND EXISTS (
            SELECT 1 
            FROM sys.database_principals rp
            WHERE rp.principal_id = rm.role_principal_id 
            AND rp.name IN ('Emperor', 'Consul', 'Tribune')
        )
    )
    BEGIN
        RAISERROR('Access denied. Requires Emperor, Consul, or Tribune role.', 16, 1);
        RETURN;
    END
    
    -- Update population records
    UPDATE Provincial.Population
    SET CurrentPopulation = @Population,
        LastCensusDate = @CensusDate,
        UpdatedBy = SYSTEM_USER,
        UpdatedDate = GETDATE()
    WHERE ProvinceID = @ProvinceID;
END;
GO

-- 5. User management procedures
CREATE OR ALTER PROCEDURE dbo.AssignRomanRole
    @UserName NVARCHAR(128),
    @RoleName NVARCHAR(128)
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Security check - only Emperor and Consul can assign roles
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.database_principals dp
        JOIN sys.database_role_members rm ON dp.principal_id = rm.member_principal_id
        WHERE dp.name = SYSTEM_USER 
        AND EXISTS (
            SELECT 1 
            FROM sys.database_principals rp
            WHERE rp.principal_id = rm.role_principal_id 
            AND rp.name IN ('Emperor', 'Consul')
        )
    )
    BEGIN
        RAISERROR('Access denied. Only Emperor or Consul can assign roles.', 16, 1);
        RETURN;
    END
    
    -- Validate role
    IF NOT EXISTS (SELECT 1 FROM sys.database_principals WHERE name = @RoleName AND type = 'R')
    BEGIN
        RAISERROR('Invalid role specified.', 16, 1);
        RETURN;
    END
    
    -- Add user to role
    DECLARE @SQL NVARCHAR(MAX);
    SET @SQL = 'ALTER ROLE [' + @RoleName + '] ADD MEMBER [' + @UserName + '];';
    EXEC sp_executesql @SQL;
    
    -- Log role assignment
    INSERT INTO dbo.RoleAssignmentLog (
        UserName,
        RoleName,
        AssignedBy,
        AssignmentDate
    )
    VALUES (
        @UserName,
        @RoleName,
        SYSTEM_USER,
        GETDATE()
    );
END;
GO

-- 6. Audit procedures
CREATE OR ALTER PROCEDURE dbo.AuditUserAccess
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Only Emperor and Censor can run audits
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.database_principals dp
        JOIN sys.database_role_members rm ON dp.principal_id = rm.member_principal_id
        WHERE dp.name = SYSTEM_USER 
        AND EXISTS (
            SELECT 1 
            FROM sys.database_principals rp
            WHERE rp.principal_id = rm.role_principal_id 
            AND rp.name IN ('Emperor', 'Censor')
        )
    )
    BEGIN
        RAISERROR('Access denied. Only Emperor or Censor can run audits.', 16, 1);
        RETURN;
    END
    
    -- Generate audit report
    SELECT 
        dp.name AS UserName,
        r.name AS RoleName,
        p.permission_name AS Permission,
        p.state_desc AS PermissionState,
        OBJECT_SCHEMA_NAME(p.major_id) AS SchemaName,
        OBJECT_NAME(p.major_id) AS ObjectName
    FROM sys.database_principals dp
    LEFT JOIN sys.database_role_members rm 
        ON dp.principal_id = rm.member_principal_id
    LEFT JOIN sys.database_principals r 
        ON rm.role_principal_id = r.principal_id
    LEFT JOIN sys.database_permissions p 
        ON dp.principal_id = p.grantee_principal_id
    WHERE dp.type IN ('S', 'U')
    ORDER BY dp.name, r.name;
END;
GO

-- 7. Example user creation and role assignment
-- Create a new Quaestor
CREATE LOGIN [Quintus] WITH PASSWORD = 'StrongP@ssw0rd';
CREATE USER [Quintus] FOR LOGIN [Quintus];
EXEC dbo.AssignRomanRole 'Quintus', 'Quaestor';

-- Create a new Tribune
CREATE LOGIN [Marcus] WITH PASSWORD = 'StrongP@ssw0rd';
CREATE USER [Marcus] FOR LOGIN [Marcus];
EXEC dbo.AssignRomanRole 'Marcus', 'Tribune';
CREATE TABLE Provinces (
    ProvinceID INT PRIMARY KEY,
    ProvinceName NVARCHAR(100),
    Governor NVARCHAR(100),
    TaxRevenue DECIMAL(18,2),
    LastUpdated DATETIME
)

CREATE TABLE TaxCollection (
    CollectionID INT PRIMARY KEY,
    ProvinceID INT,
    Amount DECIMAL(18,2),
    CollectionDate DATE,
    FOREIGN KEY (ProvinceID) REFERENCES Provinces(ProvinceID)
)
GO

-- 1. Basic Transaction Example
CREATE OR ALTER PROCEDURE dbo.CollectProvincialTax
    @ProvinceID INT,
    @Amount DECIMAL(18,2)
AS
BEGIN
    SET NOCOUNT ON;
    
    BEGIN TRY
        BEGIN TRANSACTION;
            
        -- Insert tax collection record
        INSERT INTO TaxCollection (ProvinceID, Amount, CollectionDate)
        VALUES (@ProvinceID, @Amount, GETDATE());
            
        -- Update province total
        UPDATE Provinces
        SET TaxRevenue = TaxRevenue + @Amount,
            LastUpdated = GETDATE()
        WHERE ProvinceID = @ProvinceID;
            
        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;
            
        -- Log error details
        INSERT INTO ErrorLog (
            ErrorTime,
            ErrorNumber,
            ErrorSeverity,
            ErrorState,
            ErrorProcedure,
            ErrorLine,
            ErrorMessage
        )
        VALUES (
            GETDATE(),
            ERROR_NUMBER(),
            ERROR_SEVERITY(),
            ERROR_STATE(),
            ERROR_PROCEDURE(),
            ERROR_LINE(),
            ERROR_MESSAGE()
        );
            
        -- Re-throw error
        THROW;
    END CATCH
END;

-- 2. Different Transaction Isolation Levels
-- Read Committed (Default)
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
BEGIN TRANSACTION;
    SELECT * FROM Provinces WITH (UPDLOCK)
    WHERE ProvinceID = 1;
    -- Do some work
COMMIT;

-- Repeatable Read
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
BEGIN TRANSACTION;
    SELECT * FROM Provinces
    WHERE TaxRevenue > 1000000;
    -- Other transactions cannot modify selected rows
    -- Do some work
COMMIT;

-- Serializable
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
BEGIN TRANSACTION;
    SELECT * FROM Provinces
    WHERE TaxRevenue BETWEEN 100000 AND 500000;
    -- Other transactions cannot insert rows in this range
    -- Do some work
COMMIT;

-- Snapshot
-- First, enable snapshot isolation
ALTER DATABASE RomanEmpire
SET ALLOW_SNAPSHOT_ISOLATION ON;

SET TRANSACTION ISOLATION LEVEL SNAPSHOT;
BEGIN TRANSACTION;
    -- Reads data as it existed at start of transaction
    SELECT * FROM Provinces;
    -- Do some work
COMMIT;

-- 3. Lock Hints
-- UPDLOCK - Lock rows for intended update
SELECT * FROM Provinces WITH (UPDLOCK)
WHERE ProvinceID = 1;

-- ROWLOCK - Force row-level locking
UPDATE Provinces WITH (ROWLOCK)
SET TaxRevenue = TaxRevenue + 1000
WHERE ProvinceID = 1;

-- TABLOCK - Table level lock
INSERT INTO TaxCollection WITH (TABLOCK)
SELECT ProvinceID, TaxRevenue * 0.1, GETDATE()
FROM Provinces;
GO
-- 4. Deadlock Priority
CREATE OR ALTER PROCEDURE dbo.UpdateProvinceRevenue
    @ProvinceID INT,
    @Amount DECIMAL(18,2)
AS
BEGIN
    SET DEADLOCK_PRIORITY HIGH;
    
    BEGIN TRY
        BEGIN TRANSACTION;
        
        -- Update province
        UPDATE Provinces WITH (ROWLOCK)
        SET TaxRevenue = TaxRevenue + @Amount
        WHERE ProvinceID = @ProvinceID;
        
        -- Insert collection record
        INSERT INTO TaxCollection (ProvinceID, Amount, CollectionDate)
        VALUES (@ProvinceID, @Amount, GETDATE());
        
        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        IF ERROR_NUMBER() = 1205 -- Deadlock error
        BEGIN
            -- Log deadlock
            INSERT INTO DeadlockLog (
                DeadlockTime,
                ProcedureName,
                Parameters
            )
            VALUES (
                GETDATE(),
                'UpdateProvinceRevenue',
                CONCAT('ProvinceID: ', @ProvinceID, ', Amount: ', @Amount)
            );
            
            -- Retry logic
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION;
                
            -- Wait briefly before retry
            WAITFOR DELAY '00:00:01';
            
            -- Retry the transaction once
            BEGIN TRANSACTION;
            
            UPDATE Provinces WITH (ROWLOCK)
            SET TaxRevenue = TaxRevenue + @Amount
            WHERE ProvinceID = @ProvinceID;
            
            INSERT INTO TaxCollection (ProvinceID, Amount, CollectionDate)
            VALUES (@ProvinceID, @Amount, GETDATE());
            
            COMMIT TRANSACTION;
        END
        ELSE
        BEGIN
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION;
            THROW;
        END
    END CATCH
END;

-- 5. Monitor Locks
-- Current locks in database
SELECT 
    DB_NAME(resource_database_id) AS DatabaseName,
    OBJECT_NAME(resource_associated_entity_id) AS TableName,
    request_mode AS LockType,
    request_status AS LockStatus,
    request_session_id AS SessionID,
    resource_description AS Resource
FROM sys.dm_tran_locks
WHERE resource_database_id = DB_ID('RomanEmpire');
GO

-- 6. Application Lock Example
CREATE OR ALTER PROCEDURE dbo.ProcessDailyTaxCollection
AS
BEGIN
    DECLARE @LockResult INT;
    
    -- Try to acquire application lock
    EXEC @LockResult = sp_getapplock
        @Resource = 'DailyTaxCollection',
        @LockMode = 'Exclusive',
        @LockOwner = 'Transaction',
        @LockTimeout = 30000; -- 30 seconds
        
    IF @LockResult >= 0
    BEGIN
        BEGIN TRY
            BEGIN TRANSACTION;
            
            -- Process daily collections
            INSERT INTO TaxCollection (ProvinceID, Amount, CollectionDate)
            SELECT 
                ProvinceID,
                TaxRevenue * 0.01, -- Daily rate
                GETDATE()
            FROM Provinces
            WHERE LastUpdated < DATEADD(day, -1, GETDATE());
            
            -- Update provinces
            UPDATE p
            SET TaxRevenue = TaxRevenue - (TaxRevenue * 0.01),
                LastUpdated = GETDATE()
            FROM Provinces p
            WHERE LastUpdated < DATEADD(day, -1, GETDATE());
            
            COMMIT TRANSACTION;
            
            -- Release application lock
            EXEC sp_releaseapplock
                @Resource = 'DailyTaxCollection',
                @LockOwner = 'Transaction';
        END TRY
        BEGIN CATCH
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION;
                
            -- Release application lock
            EXEC sp_releaseapplock
                @Resource = 'DailyTaxCollection',
                @LockOwner = 'Transaction';
                
            -- Rethrow the error using RAISERROR
            DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE(),
                    @ErrorSeverity INT = ERROR_SEVERITY(),
                    @ErrorState INT = ERROR_STATE();
            
            RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);
        END CATCH
    END
    ELSE
    BEGIN
        -- Handle lock timeout using RAISERROR
        RAISERROR('Failed to acquire application lock for daily tax collection', 16, 1);
    END
END;

-- 7. Monitor Transaction Status
SELECT 
    session_id,
    transaction_id,
    database_id,
    database_transaction_begin_time,
    database_transaction_log_bytes_used,
    database_transaction_log_bytes_reserved
FROM sys.dm_tran_database_transactions;
GO

-- 8. Read Past Locking
CREATE OR ALTER PROCEDURE dbo.GetProvinceTaxReport
AS
BEGIN
    -- Use READPAST to skip locked rows
    SELECT p.ProvinceName,
           p.TaxRevenue,
           t.CollectionDate,
           t.Amount
    FROM Provinces p WITH (READPAST)
    LEFT JOIN TaxCollection t WITH (READPAST)
        ON p.ProvinceID = t.ProvinceID
    WHERE t.CollectionDate >= DATEADD(month, -1, GETDATE())
    ORDER BY p.ProvinceName, t.CollectionDate;
END;

In [ ]:
-- Enable spatial features
sp_configure 'show advanced options', 1;
RECONFIGURE;
GO

-- Create spatial tables for Roman Empire
USE RomanEmpire;
GO

-- 1. Provinces (Administrative Regions)
CREATE TABLE Provinces (
    ProvinceID INT PRIMARY KEY IDENTITY(1,1),
    ProvinceName NVARCHAR(100) NOT NULL,
    YearEstablished INT,
    YearDissolved INT NULL,
    Governor NVARCHAR(100),
    Capital NVARCHAR(100),
    Population INT,
    TaxRevenue DECIMAL(18,2),
    Boundary GEOGRAPHY, -- Polygon representing province boundaries
    CentralPoint GEOGRAPHY, -- Point representing administrative center
    CONSTRAINT UQ_ProvinceName UNIQUE (ProvinceName)
);

-- 2. Cities and Settlements
CREATE TABLE Cities (
    CityID INT PRIMARY KEY IDENTITY(1,1),
    ProvinceID INT FOREIGN KEY REFERENCES Provinces(ProvinceID),
    CityName NVARCHAR(100) NOT NULL,
    CityType NVARCHAR(50) CHECK (CityType IN ('Colonia', 'Municipium', 'Civitas', 'Vicus', 'Castellum')),
    Population INT,
    YearFounded INT,
    Location GEOGRAPHY, -- Point representing city location
    CityBoundary GEOGRAPHY, -- Polygon representing city limits
    HasWalls BIT DEFAULT 0,
    HasAqueduct BIT DEFAULT 0,
    HasAmphitheater BIT DEFAULT 0,
    CONSTRAINT UQ_CityName_Province UNIQUE (CityName, ProvinceID)
);

-- 3. Roman Roads
CREATE TABLE Roads (
    RoadID INT PRIMARY KEY IDENTITY(1,1),
    RoadName NVARCHAR(100),
    RouteType NVARCHAR(50) CHECK (RouteType IN ('Via', 'Strata', 'Actus', 'Iter')),
    YearConstructed INT,
    LastMaintenance DATE,
    Route GEOGRAPHY, -- LineString representing road path
    LengthKM AS Route.STLength()/1000, -- Computed column for road length
    MaintenanceStatus NVARCHAR(20) CHECK (MaintenanceStatus IN ('Excellent', 'Good', 'Fair', 'Poor'))
);

-- 4. Military Installations
CREATE TABLE MilitaryInstallations (
    InstallationID INT PRIMARY KEY IDENTITY(1,1),
    ProvinceID INT FOREIGN KEY REFERENCES Provinces(ProvinceID),
    InstallationType NVARCHAR(50) CHECK (InstallationType IN ('Castrum', 'Castellum', 'Burgus', 'WatchTower')),
    GarrisonSize INT,
    UnitStationed NVARCHAR(100),
    Location GEOGRAPHY, -- Point representing installation location
    DefensivePerimeter GEOGRAPHY, -- Polygon representing defensive walls
    ConstructionDate INT,
    LastRenovation DATE
);

-- 5. Natural Features
CREATE TABLE NaturalFeatures (
    FeatureID INT PRIMARY KEY IDENTITY(1,1),
    FeatureName NVARCHAR(100),
    FeatureType NVARCHAR(50) CHECK (FeatureType IN ('Mountain', 'River', 'Forest', 'Lake', 'Sea')),
    Geometry GEOGRAPHY, -- Geography type based on feature (Point, LineString, or Polygon)
    Description NVARCHAR(MAX)
);

-- 6. Trade Routes
CREATE TABLE TradeRoutes (
    RouteID INT PRIMARY KEY IDENTITY(1,1),
    RouteName NVARCHAR(100),
    RouteType NVARCHAR(50) CHECK (RouteType IN ('Land', 'Maritime', 'River')),
    Path GEOGRAPHY, -- LineString representing trade route
    MainCommodities NVARCHAR(MAX),
    SeasonalityInfo NVARCHAR(MAX)
);

-- 7. Historical Events
CREATE TABLE HistoricalEvents (
    EventID INT PRIMARY KEY IDENTITY(1,1),
    EventName NVARCHAR(100),
    EventType NVARCHAR(50),
    EventDate DATE,
    Location GEOGRAPHY, -- Point or Polygon representing event location
    Description NVARCHAR(MAX)
);

-- Spatial Indexes
CREATE SPATIAL INDEX [SI_Provinces_Boundary] ON Provinces(Boundary)
USING GEOMETRY_GRID WITH (
    BOUNDING_BOX = (xmin=-20, ymin=20, xmax=50, ymax=60),
    GRIDS = (LEVEL_1 = MEDIUM, LEVEL_2 = MEDIUM, LEVEL_3 = MEDIUM, LEVEL_4 = MEDIUM),
    PAD_INDEX = ON
);

CREATE SPATIAL INDEX [SI_Cities_Location] ON Cities(Location)
USING GEOMETRY_GRID WITH (
    BOUNDING_BOX = (xmin=-20, ymin=20, xmax=50, ymax=60),
    GRIDS = (LEVEL_1 = MEDIUM, LEVEL_2 = MEDIUM, LEVEL_3 = MEDIUM, LEVEL_4 = MEDIUM),
    PAD_INDEX = ON
);

CREATE SPATIAL INDEX [SI_Roads_Route] ON Roads(Route)
USING GEOMETRY_GRID WITH (
    BOUNDING_BOX = (xmin=-20, ymin=20, xmax=50, ymax=60),
    GRIDS = (LEVEL_1 = MEDIUM, LEVEL_2 = MEDIUM, LEVEL_3 = MEDIUM, LEVEL_4 = MEDIUM),
    PAD_INDEX = ON
);
GO
-- Useful Spatial Functions

-- 1. Calculate Distance Between Cities
CREATE OR ALTER FUNCTION dbo.CalculateRouteMileage
(
    @CityID1 INT,
    @CityID2 INT
)
RETURNS DECIMAL(10,2)
AS
BEGIN
    DECLARE @Distance DECIMAL(10,2);
    
    SELECT @Distance = c1.Location.STDistance(c2.Location)/1000
    FROM Cities c1
    CROSS JOIN Cities c2
    WHERE c1.CityID = @CityID1
    AND c2.CityID = @CityID2;
    
    RETURN @Distance;
END;
GO

-- 2. Find Cities Within Range
CREATE OR ALTER PROCEDURE dbo.FindCitiesWithinRange
    @Latitude DECIMAL(9,6),
    @Longitude DECIMAL(9,6),
    @RangeKM INT
AS
BEGIN
    DECLARE @CenterPoint GEOGRAPHY;
    SET @CenterPoint = GEOGRAPHY::Point(@Latitude, @Longitude, 4326);
    
    SELECT 
        c.CityName,
        p.ProvinceName,
        c.Population,
        c.Location.STDistance(@CenterPoint)/1000 AS DistanceKM
    FROM Cities c
    JOIN Provinces p ON c.ProvinceID = p.ProvinceID
    WHERE c.Location.STDistance(@CenterPoint) <= @RangeKM * 1000
    ORDER BY c.Location.STDistance(@CenterPoint);
END;
GO

-- 3. Find Nearest Military Installation
CREATE OR ALTER PROCEDURE dbo.FindNearestMilitaryInstallation
    @CityID INT
AS
BEGIN
    DECLARE @CityLocation GEOGRAPHY;
    
    SELECT @CityLocation = Location
    FROM Cities
    WHERE CityID = @CityID;
    
    SELECT TOP 1
        m.InstallationType,
        m.GarrisonSize,
        m.UnitStationed,
        m.Location.STDistance(@CityLocation)/1000 AS DistanceKM
    FROM MilitaryInstallations m
    ORDER BY m.Location.STDistance(@CityLocation);
END;
GO

-- 4. Calculate Province Area
CREATE OR ALTER FUNCTION dbo.CalculateProvinceArea
(
    @ProvinceID INT
)
RETURNS DECIMAL(10,2)
AS
BEGIN
    DECLARE @AreaKM2 DECIMAL(10,2);
    
    SELECT @AreaKM2 = Boundary.STArea()/1000000
    FROM Provinces
    WHERE ProvinceID = @ProvinceID;
    
    RETURN @AreaKM2;
END;
GO

-- 5. Find Intersecting Trade Routes
CREATE OR ALTER PROCEDURE dbo.FindIntersectingTradeRoutes
    @RouteID INT
AS
BEGIN
    DECLARE @RoutePath GEOGRAPHY;
    
    SELECT @RoutePath = Path
    FROM TradeRoutes
    WHERE RouteID = @RouteID;
    
    SELECT 
        t.RouteName,
        t.RouteType,
        t.MainCommodities,
        t.Path.STIntersection(@RoutePath).STAsText() AS IntersectionPoint
    FROM TradeRoutes t
    WHERE t.RouteID != @RouteID
    AND t.Path.STIntersects(@RoutePath) = 1;
END;
GO

-- Example Queries

-- 1. Find all cities within 100km of Rome
EXEC dbo.FindCitiesWithinRange 41.9028, 12.4964, 100;

-- 2. Calculate the total length of roads in a province
SELECT 
    p.ProvinceName,
    SUM(r.LengthKM) AS TotalRoadLengthKM
FROM Provinces p
JOIN Cities c ON p.ProvinceID = c.ProvinceID
JOIN Roads r ON r.Route.STIntersects(p.Boundary) = 1
GROUP BY p.ProvinceName;

-- 3. Find provinces with the most military installations
SELECT 
    p.ProvinceName,
    COUNT(m.InstallationID) AS InstallationCount,
    SUM(m.GarrisonSize) AS TotalGarrisonSize
FROM Provinces p
JOIN MilitaryInstallations m ON p.ProvinceID = m.ProvinceID
GROUP BY p.ProvinceName
ORDER BY InstallationCount DESC;

-- 4. Analyze trade route density
SELECT 
    p.ProvinceName,
    COUNT(DISTINCT t.RouteID) AS TradeRouteCount,
    SUM(t.Path.STLength())/1000 AS TotalTradeRouteLengthKM
FROM Provinces p
JOIN TradeRoutes t ON t.Path.STIntersects(p.Boundary) = 1
GROUP BY p.ProvinceName
ORDER BY TradeRouteCount DESC;

In [ ]:
-- Enable system versioning at database level
ALTER DATABASE RomanEmpire SET ALLOW_SNAPSHOT_ISOLATION ON;
GO

-- 1. Province Administration Temporal Table
CREATE TABLE Provinces
(
    ProvinceID INT IDENTITY(1,1) PRIMARY KEY,
    ProvinceName NVARCHAR(100) NOT NULL,
    Governor NVARCHAR(100),
    Capital NVARCHAR(100),
    Population INT,
    TaxRevenue DECIMAL(18,2),
    Status NVARCHAR(20),
    ValidFrom DATETIME2 GENERATED ALWAYS AS ROW START,
    ValidTo DATETIME2 GENERATED ALWAYS AS ROW END,
    PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)
)
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.ProvincesHistory));

-- 2. Military Legion Assignments
CREATE TABLE LegionAssignments
(
    LegionID INT,
    ProvinceID INT,
    CommandingOfficer NVARCHAR(100),
    StrengthCount INT,
    BaseLocation NVARCHAR(100),
    Status NVARCHAR(20),
    ValidFrom DATETIME2 GENERATED ALWAYS AS ROW START,
    ValidTo DATETIME2 GENERATED ALWAYS AS ROW END,
    PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo),
    PRIMARY KEY (LegionID, ProvinceID),
    FOREIGN KEY (ProvinceID) REFERENCES Provinces(ProvinceID)
)
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.LegionAssignmentsHistory));

-- 3. Provincial Tax Rates
CREATE TABLE TaxRates
(
    ProvinceID INT,
    TaxType NVARCHAR(50),
    Rate DECIMAL(5,2),
    Description NVARCHAR(MAX),
    ValidFrom DATETIME2 GENERATED ALWAYS AS ROW START,
    ValidTo DATETIME2 GENERATED ALWAYS AS ROW END,
    PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo),
    PRIMARY KEY (ProvinceID, TaxType),
    FOREIGN KEY (ProvinceID) REFERENCES Provinces(ProvinceID)
)
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.TaxRatesHistory));

-- 4. Administrative Positions
CREATE TABLE AdministrativePositions
(
    PositionID INT IDENTITY(1,1) PRIMARY KEY,
    ProvinceID INT,
    PositionTitle NVARCHAR(100),
    OfficialName NVARCHAR(100),
    Rank INT,
    Status NVARCHAR(20),
    ValidFrom DATETIME2 GENERATED ALWAYS AS ROW START,
    ValidTo DATETIME2 GENERATED ALWAYS AS ROW END,
    PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo),
    FOREIGN KEY (ProvinceID) REFERENCES Provinces(ProvinceID)
)
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.AdministrativePositionsHistory));

-- 5. Stored Procedures for Temporal Analysis
GO
-- Get Province History
CREATE OR ALTER PROCEDURE dbo.GetProvinceHistory
    @ProvinceID INT,
    @StartDate DATETIME2,
    @EndDate DATETIME2
AS
BEGIN
    SELECT 
        ProvinceID,
        ProvinceName,
        Governor,
        Capital,
        Population,
        TaxRevenue,
        Status,
        ValidFrom,
        ValidTo
    FROM Provinces
    FOR SYSTEM_TIME BETWEEN @StartDate AND @EndDate
    WHERE ProvinceID = @ProvinceID
    ORDER BY ValidFrom;
END;
GO

-- Track Governor Changes
CREATE OR ALTER PROCEDURE dbo.TrackGovernorChanges
    @ProvinceID INT
AS
BEGIN
    SELECT 
        ProvinceID,
        Governor,
        ValidFrom AS AppointmentDate,
        ValidTo AS EndDate,
        DATEDIFF(DAY, ValidFrom, ValidTo) AS DaysInOffice
    FROM Provinces
    FOR SYSTEM_TIME ALL
    WHERE ProvinceID = @ProvinceID
        AND Governor IS NOT NULL
    ORDER BY ValidFrom;
END;
GO

-- Analyze Legion Movements
CREATE OR ALTER PROCEDURE dbo.AnalyzeLegionMovements
    @LegionID INT,
    @StartDate DATETIME2,
    @EndDate DATETIME2
AS
BEGIN
    SELECT 
        l.LegionID,
        p.ProvinceName,
        l.CommandingOfficer,
        l.StrengthCount,
        l.BaseLocation,
        l.Status,
        l.ValidFrom AS MovementDate,
        l.ValidTo AS EndDate
    FROM LegionAssignments FOR SYSTEM_TIME BETWEEN @StartDate AND @EndDate l
    JOIN Provinces p ON l.ProvinceID = p.ProvinceID
    WHERE l.LegionID = @LegionID
    ORDER BY l.ValidFrom;
END;
GO


-- Compare Tax Rates Over Time
CREATE OR ALTER PROCEDURE dbo.CompareTaxRates
    @ProvinceID INT,
    @TaxType NVARCHAR(50),
    @CompareDate1 DATETIME2,
    @CompareDate2 DATETIME2
AS
BEGIN
    SELECT 
        p.ProvinceName,
        t1.TaxType,
        t1.Rate AS Rate1,
        t2.Rate AS Rate2,
        t1.ValidFrom AS Date1,
        t2.ValidFrom AS Date2,
        (t2.Rate - t1.Rate) AS RateChange
    FROM Provinces p
    CROSS APPLY (
        SELECT TOP 1 * 
        FROM TaxRates FOR SYSTEM_TIME AS OF @CompareDate1
        WHERE ProvinceID = @ProvinceID AND TaxType = @TaxType
    ) t1
    CROSS APPLY (
        SELECT TOP 1 * 
        FROM TaxRates FOR SYSTEM_TIME AS OF @CompareDate2
        WHERE ProvinceID = @ProvinceID AND TaxType = @TaxType
    ) t2
    WHERE p.ProvinceID = @ProvinceID;
END;
GO

-- Administrative Timeline Analysis
CREATE OR ALTER PROCEDURE dbo.AnalyzeAdministrativeTimeline
    @ProvinceID INT,
    @StartDate DATETIME2,
    @EndDate DATETIME2
AS
BEGIN
    SELECT 
        p.ProvinceName,
        ap.PositionTitle,
        ap.OfficialName,
        ap.Rank,
        ap.Status,
        ap.ValidFrom AS AppointmentDate,
        ap.ValidTo AS EndDate,
        DATEDIFF(DAY, ap.ValidFrom, ap.ValidTo) AS DaysInPosition
    FROM AdministrativePositions FOR SYSTEM_TIME BETWEEN @StartDate AND @EndDate ap
    JOIN Provinces p ON ap.ProvinceID = p.ProvinceID
    WHERE ap.ProvinceID = @ProvinceID
    ORDER BY ap.Rank, ap.ValidFrom;
END;
GO

-- 6. Example Usage

-- Insert sample province
INSERT INTO Provinces 
(ProvinceName, Governor, Capital, Population, TaxRevenue, Status)
VALUES 
('Hispania', 'Gaius Julius Caesar', 'Tarraco', 1500000, 1000000.00, 'Active');

-- Update governor
UPDATE Provinces 
SET Governor = 'Marcus Aemilius Lepidus'
WHERE ProvinceName = 'Hispania';

-- Query history as of a specific date
DECLARE @HistoricalDate DATETIME2 = '0045-03-15';
SELECT *
FROM Provinces
FOR SYSTEM_TIME AS OF @HistoricalDate
WHERE ProvinceName = 'Hispania';

-- Query changes over a time period
DECLARE @StartDate DATETIME2 = '0045-01-01';
DECLARE @EndDate DATETIME2 = '0045-12-31';
SELECT *
FROM Provinces
FOR SYSTEM_TIME BETWEEN @StartDate AND @EndDate
WHERE ProvinceName = 'Hispania'
ORDER BY ValidFrom;
GO
-- 7. Temporal Table Maintenance

-- Cleanup historical data beyond retention period
CREATE OR ALTER PROCEDURE dbo.CleanupHistoricalData
    @RetentionYears INT
AS
BEGIN
    DECLARE @RetentionDate DATETIME2 = DATEADD(YEAR, -@RetentionYears, GETDATE());
    
    ALTER TABLE Provinces SET (SYSTEM_VERSIONING = OFF);
    DELETE FROM dbo.ProvincesHistory
    WHERE ValidTo < @RetentionDate;
    ALTER TABLE Provinces SET (SYSTEM_VERSIONING = ON);
    
    -- Repeat for other temporal tables...
END;
GO

-- Monitor temporal table growth
CREATE OR ALTER PROCEDURE dbo.MonitorTemporalStorage
AS
BEGIN
    SELECT 
        t.name AS TableName,
        p.rows AS CurrentRows,
        h.rows AS HistoryRows,
        CAST(ROUND(((ps.reserved_page_count * 8.0) / 1024), 2) AS DECIMAL(10,2)) AS CurrentSizeMB,
        CAST(ROUND(((hs.reserved_page_count * 8.0) / 1024), 2) AS DECIMAL(10,2)) AS HistorySizeMB
    FROM sys.tables t
    JOIN sys.partitions p ON t.object_id = p.object_id
    JOIN sys.tables ht ON t.history_table_id = ht.object_id
    JOIN sys.partitions h ON ht.object_id = h.object_id
    JOIN sys.dm_db_partition_stats ps ON t.object_id = ps.object_id
    JOIN sys.dm_db_partition_stats hs ON ht.object_id = hs.object_id
    WHERE t.temporal_type = 2;
END;
GO